
---

# 🧩 **Parsing Tool Results** – *How to Handle and Use Output from MCP Tools*

---

## 📌 What It Does

When you call a tool using `MCPClient`, it returns a structured result — **ToolOutput** — which includes not just the result, but also metadata like status, error info, and output type.

You can **parse** this result to:

* Extract usable data (e.g., strings, lists, dicts)
* Handle errors properly
* Enable downstream processing (e.g., chaining results)

---

## ✅ Why Parse Tool Results?

| Scenario                         | Why Parsing Matters                        |
| -------------------------------- | ------------------------------------------ |
| 🧾 You need only the core output | Result is wrapped — must extract `.output` |
| ❌ Tool may fail                  | You should handle `.success == False`      |
| 🔗 Output used by next tool      | Parse it before sending forward            |

---

## 🧠 Tool Output Structure

```python
ToolOutput(
    output: Any,
    success: bool,
    error: Optional[str] = None,
    output_type: Optional[str] = None
)
```

### 🔍 Accessing fields:

```python
result = client.call_tool("summarize", {"text": "some long text"})

print(result.output)       # 👈 Final result (e.g., summary string)
print(result.success)      # ✅ True if tool ran without error
print(result.error)        # ⚠️ None if no error, else error string
print(result.output_type)  # 🏷️ E.g., "text", "json", "float"
```

---

## 📦 Common Output Types & Parsing Tips

| `output_type` | Type in Python | Example Use                          |
| ------------- | -------------- | ------------------------------------ |
| `"text"`      | `str`          | For summaries, generated text        |
| `"json"`      | `dict`         | Structured info — parse keys easily  |
| `"list"`      | `list`         | Multi-result tools — loop over items |
| `"float"`     | `float`        | Score, confidence, similarity, etc.  |

---

## ✅ Recommended Parsing Pattern

```python
def run_tool_and_parse(client, tool_name, input_dict):
    result = client.call_tool(tool_name, input_dict)
    if result.success:
        return result.output
    else:
        print(f"Error: {result.error}")
        return None
```

Use this to **safely call and parse** any tool with fallback handling.

---

## 🧠 Real-Time Use Case: Chaining Tools

```python
def smart_summarize_and_translate(text):
    summary = client.call_tool("summarize", {"text": text})
    if not summary.success:
        return {"error": summary.error}
    
    translated = client.call_tool("translate", {
        "text": summary.output,
        "lang": "es"
    })
    
    return {
        "summary": summary.output,
        "translated": translated.output if translated.success else None
    }
```

---

## ⚠️ Limitations & Tips

* ❗ `.output` is `None` if `success == False`
* 🔄 If `.output_type == "json"` but `output` is `str`, manually `json.loads()`
* 🧪 Always check `output_type` if using dynamically chained tools

---

## 🧩 Summary

| Feature        | Description                            |
| -------------- | -------------------------------------- |
| `ToolOutput`   | Unified format for all tool responses  |
| `.output`      | Final result (value you need)          |
| `.success`     | Boolean flag for result validity       |
| `.error`       | Error message string if failure        |
| `.output_type` | Optional string tag (text, json, etc.) |

---

